In [1]:
import gctree
import glob
import numpy as np
import pandas as pd
import pickle
from Bio.Seq import Seq
from collections import defaultdict
from gctree.branching_processes import CollapsedTree

## Load GC trees

From Will's `phenotype-trajectories.ipynb`

In [2]:
results = "../results/latest"

def gc_key(file):
    PR_str = file.split("/")[-2].split("-")
    return int(PR_str[1]), f"{PR_str[3]}-{PR_str[2]}"

files = sorted(glob.glob(f"{results}/gctrees/PR*/gctree.p"), key=gc_key)

original_trees = {}

for file in files:
    key = "_".join(f"{x}{y}" for x, y in zip(("mouse", "GC"), gc_key(file)))
    assert key not in original_trees, key
    original_trees[key] = pickle.load(open(file, "rb"))

print(f"Loaded {len(original_trees)} GC trees")

Loaded 95 GC trees


## Generating sequence data from these trees

Two ideas so far...

### 1 | Simulate trees by sampling mutations along the same tree topology

Pros:

- Provides sequence data at the nodes

Cons:

- We only sample trees of the same structure
- The proposed summary statistic (correlation between abundances and num mutations) won't vary

Simulating trees (from Will's `phenotype-trajectories.ipynb`, slightly modified the loop to remove the rejection scheme):

In [3]:
mutability = pd.read_csv("../../nextflow/data/mutability/MK_RS5NF_mutability.csv", sep=" ", index_col=0).squeeze("columns")
substitution = pd.read_csv("../../nextflow/data/mutability/MK_RS5NF_substitution.csv", sep=" ", index_col=0)

naive = next(iter(original_trees.values())).tree.sequence
igh_frame = 1
igk_frame = 1
igk_idx = 336

def mutate_uniform(sequence):
    i = np.random.choice(len(sequence))
    base = sequence[i]
    alt_base = np.random.choice(list("ACGT".replace(base, "")))
    sequence = list(sequence)
    sequence[i] = alt_base
    return "".join(sequence)

def mutate_S5F(sequence):
    sequence_H = "NN" + sequence[:igk_idx] + "NN"
    sequence_K = "NN" + sequence[igk_idx:] + "NN"
    # mutabilities of each nucleotide
    contexts = [sequence_H[(i - 2) : (i + 3)]
                    for i in range(2, len(sequence_H) - 2)
                   ] + [sequence_K[(i - 2) : (i + 3)]
                        for i in range(2, len(sequence_K) - 2)
                       ]
    mutabilities = np.array([mutability[context] for context in contexts])
    i = np.random.choice(len(mutabilities), p=mutabilities / sum(mutabilities))
    sequence = sequence[:i] + np.random.choice(substitution.columns, p=substitution.loc[contexts[i]].fillna(0)) + sequence[(i + 1):]
    return sequence

def aa(seq, frame):
    return Seq(seq[(frame - 1) : (frame - 1 + (3 * ((len(seq) - (frame - 1)) // 3)))]).translate()

def mutations(naive_aa, aa, pos_map, chain_annotation):
    return [f"{aa1}{pos_map[pos]}{chain_annotation}{aa2}"
            for pos, (aa1, aa2) in enumerate(zip(naive_aa, aa))
            if aa1 != aa2]

naive_igh_aa = aa(naive[:igk_idx], igh_frame)
naive_igk_aa = aa(naive[igk_idx:], igk_frame)

pos_df = pd.read_csv("https://raw.githubusercontent.com/jbloomlab/Ab-CGGnaive_DMS/main/data/CGGnaive_sites.csv", dtype=dict(site=pd.Int16Dtype()), index_col="site_scFv")

igh_pos_map = pos_df.loc[pos_df.chain == "H", "site"].reset_index(drop=True)
igk_pos_map = pos_df.loc[pos_df.chain == "L", "site"].reset_index(drop=True)

dms_df = pd.read_csv("https://media.githubusercontent.com/media/jbloomlab/Ab-CGGnaive_DMS/main/results/final_variant_scores/final_variant_scores.csv", index_col="mutation", dtype=dict(position_IMGT=pd.Int16Dtype()))
# remove linker sites
dms_df = dms_df[dms_df.chain != "link"]
# add indicator for wildtype data
dms_df["WT"] = dms_df.wildtype == dms_df.mutant
assert dms_df.position_IMGT.max() < 1000
dms_df["site"] = [f"{chain}-{str(pos).zfill(3)}" for chain, pos in zip(dms_df.chain, dms_df.position_IMGT)]

In [4]:
#| output: false

sim_trees = defaultdict(list)
expression_threshold = 0.2
n_replicates = 1 # TODO: increase this back to 20
for gc, tree in original_trees.items():
    replicate = 1
    dead = False
    while replicate <= n_replicates:
        print(f"GC {gc} replicate {replicate}", end="   \r")
        new_tree = tree.tree.copy()
        for node, node_new in zip(tree.tree.traverse(strategy="preorder"), new_tree.traverse(strategy="preorder")):
            assert node.name == node_new.name
            if not node.is_root():
                success = False
                closest = np.inf

                node_new.sequence = node_new.up.sequence
                for _ in range(int(node.dist)):
                    node_new.sequence = mutate_S5F(node_new.sequence)
                    igh_aa = aa(node_new.sequence[:igk_idx], igh_frame)
                    igk_aa = aa(node_new.sequence[igk_idx:], igk_frame)
                    igh_mutations = mutations(naive_igh_aa, igh_aa, igh_pos_map, "(H)")
                    igk_mutations = mutations(naive_igk_aa, igk_aa, igk_pos_map, "(L)")        
                    all_mutations = igh_mutations + igk_mutations
                    if any("*" in x for x in all_mutations):
                        node_new.delta_bind_CGG_FVS_additive = None
                        node_new.delta_expr_FVS_additive = None
                    else:
                        node_new.delta_bind_CGG_FVS_additive = dms_df.delta_bind_CGG[all_mutations].sum()
                        node_new.delta_expr_FVS_additive = dms_df.delta_expr[all_mutations].sum()

        sim_trees[gc].append(new_tree)
        replicate += 1

Extracting sequences from simulated trees:

In [5]:
sim_sequences = []

for gc, trees in sim_trees.items():
    for i, tree in enumerate(trees):
        abundances = {node.name:node.abundance for node in tree.traverse()}
        abundances["gc"] = gc
        abundances["replicate"] = i+1
        sim_sequences.append(abundances)

sim_sequences = pd.DataFrame(sim_sequences)

# Reorder the columns
first_cols = ["gc", "replicate", "naive"]
last_cols = sorted(col for col in sim_sequences.columns if col not in first_cols)
sim_sequences = sim_sequences[first_cols + last_cols]

sim_sequences

,gc,replicate,naive,1,10,11,12,13,14,15,...,seq81,seq82,seq83,seq84,seq85,seq86,seq87,seq88,seq89,seq9
0,mouse1_GC1-RP,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,mouse1_GC11-RB,1,0,0,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,mouse1_GC12-RB,1,0,0,0.0,0.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,mouse1_GC13-RB,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,mouse1_GC14-RB,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,mouse12_GC126-RB,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
91,mouse12_GC127-RB,1,0,0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
92,mouse12_GC128-LB,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
93,mouse12_GC129-LB,1,0,0,0.0,0.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


Need to compute summary stats

### 2 | Simulate trees by sampling the Galton-Watson process with the same $p, q$

Pros:

- We will get varying tree topologies

Cons:

- Might need to do a rejection scheme until we sample a tree with enough cells
- No guarantee the sampled tree will be as highly parsimonious
- No information about sequences, just abstract types that need to be filled in

In [6]:
#| output: false

sim_trees2 = defaultdict(list)

for gc, tree in original_trees.items():
    p, q = tree.mle()

    for replicate in range(1, n_replicates+1):
        print(f"GC {gc} replicate {replicate}", end="   \r")

        proposed_tree = CollapsedTree()

        # TODO: this is just a simple check to avoid very very simple trees (length 1)
        while (not proposed_tree.tree or len(proposed_tree.tree) <= 1):
            proposed_tree.simulate(p, q)

        sim_trees2[gc].append(proposed_tree)

Need to fill in sequences info then compute summary stats